<a href="https://colab.research.google.com/github/Fackor/Advanced_Machine_Learning_Course/blob/main/Zero_Shot_Pipeline_for_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 13.4MB/s 
     |████████████████████████████████| 3.0MB 46.2MB/s 
     |████████████████████████████████| 890kB 55.7MB/s 
     |████████████████████████████████| 1.1MB 44.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=73beb5fa991aaac0e22d8c0680e4c4263977b513ad3c0273e696ee8b00f5c8c6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from transformers import pipeline

In [5]:
#classifier = pipeline("zero-shot-classification")
classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def zeroshot_NER(paragraph, candidate_labels, threshold):
  words = [word.lower() for word in paragraph.split() if word.isalpha()]
  NER = []
  for word in words:
    output = classifier(word, candidate_labels)

    label =  output['labels'][0] if output['scores'][0] > threshold else '[UNK]'
    NER.append([word, label])

  return NER  

In [ ]:
sequence = input("Enter a paragraph to do NER on.")
candidate_labels = input("Enter candidate labels separated by double colon(::).").split("::")
candidate_labels.append('[UNK]')

NER = zeroshot_NER(sequence, candidate_labels, 0.4)
for each in NER:
  print(f"{each[0]}<{each[1]}>", end=" ")

In [7]:
from sklearn.metrics import f1_score
from tqdm import tqdm

candidate_labels = ['Person', 'Organization', 'Location', 'Miscellaneous', '[UNK]']

label_maps = {'Person' : 'PER',
              'Organization' : 'ORG',
              'Location' : 'LOC',
              'Miscellaneous': 'MISC', 
              '[UNK]' : 'O'}

other_label_map = {'B-ORG' : 'ORG',
                   'I-ORG' : 'ORG',
                   'B-PER' : 'PER',
                   'I-PER' : 'PER',
                   'B-LOC' : 'LOC',
                   'I-LOC' : 'LOC',
                   'B-MISC' : 'MISC',
                   'I-MISC' : 'MISC',
                   'O' : 'O'}

filename = '/content/valid.txt'

In [8]:
result = {}

predictions = []
labels = []

THRESHOLD = 0.5
with open(filename) as f:
  for i, line in tqdm(enumerate(f)):
    if i == 0:
        continue
    line = line.split()
    if len(line) > 0 and line[0].isalpha():
      word = line[0]
      label = other_label_map[line[3]]
      labels.append(label)
      
      out = zeroshot_NER(word, candidate_labels, THRESHOLD)

      predictions.append(label_maps[out[0][1]])


for label in label_maps.values():
  pred = [x == label for x in predictions]
  true = [x == label for x in labels]
  result[label] = f1_score(true, pred, average='binary')

print(result)
'''
50350it [22:49, 36.76it/s]

{'PER': 0.2156064461407973, 'ORG': 0.036968576709796676, 'LOC': 0.3058277462609592, 'MISC': 0.005212858384013901, 'O': 0.7298355194669998}
'''

55044it [20:00, 45.85it/s]


{'PER': 0.21206682313958305, 'ORG': 0.04953560371517028, 'LOC': 0.35424354243542433, 'MISC': 0.0, 'O': 0.7353024911032029}


"\n50350it [22:49, 36.76it/s]\n\n{'PER': 0.2156064461407973, 'ORG': 0.036968576709796676, 'LOC': 0.3058277462609592, 'MISC': 0.005212858384013901, 'O': 0.7298355194669998}\n"

In [9]:
result = {}

predictions = []
labels = []

THRESHOLD = 0.6
with open(filename) as f:
  for i, line in tqdm(enumerate(f)):
    if i == 0:
        continue
    line = line.split()
    if len(line) > 0 and line[0].isalpha():
      word = line[0]
      label = other_label_map[line[3]]
      labels.append(label)
      
      #print(line)
      #print(word)
      
      out = zeroshot_NER(word, candidate_labels, THRESHOLD)

      predictions.append(label_maps[out[0][1]])


for label in label_maps.values():
  pred = [x == label for x in predictions]
  true = [x == label for x in labels]
  result[label] = f1_score(true, pred, average='binary')

print(result)
'''
55044it [20:10, 45.48it/s]

{'PER': 0.21206682313958305, 'ORG': 0.04953560371517028, 'LOC': 0.35424354243542433, 'MISC': 0.0, 'O': 0.7353024911032029}
'''

55044it [19:37, 46.74it/s]


{'PER': 0.3452311293153891, 'ORG': 0.03107861060329067, 'LOC': 0.3003725995987389, 'MISC': 0.0, 'O': 0.8625975473801561}


"\n55044it [20:10, 45.48it/s]\n\n{'PER': 0.21206682313958305, 'ORG': 0.04953560371517028, 'LOC': 0.35424354243542433, 'MISC': 0.0, 'O': 0.7353024911032029}\n"

In [10]:
result = {}

predictions = []
labels = []

THRESHOLD = 0.7
with open(filename) as f:
  for i, line in tqdm(enumerate(f)):
    if i == 0:
        continue
    line = line.split()
    if len(line) > 0 and line[0].isalpha():
      word = line[0]
      label = other_label_map[line[3]]
      labels.append(label)
      
      #print(line)
      #print(word)
      
      out = zeroshot_NER(word, candidate_labels, THRESHOLD)

      predictions.append(label_maps[out[0][1]])


for label in label_maps.values():
  pred = [x == label for x in predictions]
  true = [x == label for x in labels]
  result[label] = f1_score(true, pred, average='binary')

print(result)

55044it [20:22, 45.03it/s]


{'PER': 0.31990281433488554, 'ORG': 0.02212602212602213, 'LOC': 0.24445936870382806, 'MISC': 0.0, 'O': 0.8804367227940739}
